In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn import metrics
import time
import dataframe_image as dfi

In [2]:
#Parkinsons Dataset
df = pd.read_csv('./datasets/parkinsons.csv')
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
df_scaled['target'] = targets
columns = df_scaled.columns
dir_path = 'Parkinsons'
dataset_name = 'Parkinsons'
display(df_scaled)

0.0 1.0
Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,PPE,RPDE,DFA,spread1,spread2,D2,target
0,0.184308,0.112592,0.054815,0.195680,0.249012,0.145472,0.247588,0.145288,0.312215,0.280197,...,0.332584,0.068307,0.511745,0.497310,0.369155,0.960148,0.569875,0.585765,0.390661,1
1,0.198327,0.094930,0.278323,0.254130,0.288538,0.191233,0.323687,0.191042,0.472887,0.444536,...,0.516048,0.059331,0.432577,0.671326,0.470830,0.977024,0.703277,0.741337,0.473145,1
2,0.165039,0.059128,0.265288,0.280178,0.328063,0.229287,0.369239,0.229411,0.390634,0.326212,...,0.443317,0.039596,0.496220,0.596682,0.404416,1.000000,0.636745,0.686371,0.408819,1
3,0.165004,0.072927,0.264200,0.263342,0.328063,0.209056,0.324759,0.208862,0.414278,0.354971,...,0.475478,0.040997,0.495936,0.671949,0.416255,0.975885,0.695627,0.738089,0.436977,1
4,0.161150,0.080909,0.260107,0.354511,0.407115,0.282755,0.437299,0.282870,0.499452,0.410025,...,0.584542,0.054174,0.455499,0.757611,0.375159,0.992813,0.762472,0.513798,0.404336,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.499820,0.262986,0.165722,0.092440,0.090909,0.093931,0.089496,0.094076,0.286014,0.262942,...,0.362306,0.085909,0.450134,0.183318,0.447684,0.333127,0.257894,0.260408,0.549049,0
191,0.705488,0.307974,0.138243,0.125794,0.090909,0.126686,0.107181,0.126826,0.164050,0.146261,...,0.221338,0.055543,0.435097,0.257558,0.408567,0.434101,0.319956,0.276956,0.605474,0
192,0.502730,0.281413,0.050727,0.378653,0.288538,0.267823,0.252947,0.267940,0.123608,0.140509,...,0.156631,0.338988,0.383728,0.180580,0.352318,0.324299,0.212945,0.342577,0.558967,0
193,0.642893,0.601807,0.054279,0.181703,0.130435,0.145472,0.159700,0.145288,0.122512,0.128184,...,0.155989,0.227838,0.429936,0.163137,0.454176,0.277579,0.220650,0.452885,0.318222,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.3,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

training_data = pd.DataFrame(X_train, columns = columns[:-1])
training_data[columns[-1]] = y_train
testing_data = pd.DataFrame(X_test, columns = columns[:-1])
testing_data[columns[-1]] = y_test
dataframe = pd.concat([training_data, testing_data])
data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

original_bounds = [[dataframe[dataframe.columns[i]].min(),dataframe[dataframe.columns[i]].max()] for i in range(len(dataframe.columns[:-1]))]
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')

In [4]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

In [5]:
predictions = []
possible_classes = np.unique(y_test)
class_indexes = []
class_predictions = []
for i in range(n_classes):
    class_indexes.append([])
    class_predictions.append([])
possible_classes, class_indexes, class_predictions
data = testing_data.to_numpy()
for i in range(len(data)):
    predictions.append(mlp_explainer.model_classification_output(k_model=keras_model, net_input=data[i, :-1])[1].numpy())    
    for j,p_class in enumerate(possible_classes):
        if predictions[-1] == p_class:
            class_indexes[j].append(i)
            class_predictions[j].append(data[i, :-1])
print("Accuracy Test Data:", metrics.accuracy_score(testing_data.to_numpy()[:, -1], predictions))

Accuracy Test Data: 0.8813559322033898


In [6]:
cols = list(testing_data.columns)
if 'target' not in cols:
    cols.append('target')
predicted_dataset = []
for i,pos_class in enumerate(np.unique(y_test)):
    for instance in (testing_data.to_numpy()[:, :-1][class_indexes[i]]):
        instance = np.append(instance, pos_class.astype('int'))
        predicted_dataset.append(instance)
predicted_dataset = np.asarray(predicted_dataset)
pred_dataset_df = pd.DataFrame(predicted_dataset, columns=cols)
pred_dataset_df['target'] = pred_dataset_df['target'].astype('int')

In [7]:
metrics_dataframes = []
times_onestep = []
sizes_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []

In [8]:
original_bounds

[[0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 0.9999999999999998],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999]]

In [9]:
onestep_explanations = []
for j in range(len(pred_dataset_df['target'].unique())):
    for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
        start = time.perf_counter()
        explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,
                                                             )
        end = time.perf_counter()
        onestep_explanations.append(explanation)
        times_onestep.append(end-start)
        sizes_onestep.append(len(minimal))
        rsum_onestep.append(mymetrics.range_sum(onestep_explanations[-1]))
        coverage_onestep.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))

In [10]:
p_values = [0.50] #[0.25, 0.50, 0.75]
for p_value in p_values:
    print(f"p = {p_value}")
    times_twostep = []
    sizes_twostep = []
    rsum_twostep = []
    coverage_twostep = []
    twostep_explanations = []
    for j in range(len(pred_dataset_df['target'].unique())):
        for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
            start = time.perf_counter()
            
            explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
            end = time.perf_counter()
            twostep_explanations.append(explanation)
            times_twostep.append(end-start)
            sizes_twostep.append(len(minimal))
            rsum_twostep.append(mymetrics.range_sum(twostep_explanations[-1]))
            
            coverage_twostep.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))

    time_mean_twostep = sum(times_twostep)/len(times_twostep)
    time_std_twostep = np.std(times_twostep)
    sizes_mean_twostep = sum(sizes_twostep)/len(sizes_twostep)
    sizes_std_twostep = np.std(sizes_twostep)
    rsum_mean_twostep = sum(rsum_twostep)/len(rsum_twostep)
    rsum_std_twostep = np.std(rsum_twostep)

    coverage_mean_twostep = sum(coverage_twostep)/len(coverage_twostep)
    coverage_std_twostep = np.std(coverage_twostep)

    time_mean_onestep = sum(times_onestep)/len(times_onestep)
    time_std_onestep = np.std(times_onestep)
    sizes_mean_onestep = sum(sizes_onestep)/len(sizes_onestep)
    sizes_std_onestep = np.std(sizes_onestep)
    rsum_mean_onestep = sum(rsum_onestep)/len(rsum_onestep)
    rsum_std_onestep = np.std(rsum_onestep)

    coverage_mean_onestep = sum(coverage_onestep)/len(coverage_onestep)
    coverage_std_onestep = np.std(coverage_onestep)

    all_metrics_names = ['Metric','Onestep_MEAN','Onestep_STD','Twostep_MEAN','Twostep_STD']

    all_metrics_mean_df  = pd.DataFrame(columns=all_metrics_names)
    pattern_row = ['Time',time_mean_onestep, time_std_onestep, time_mean_twostep,time_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Size', sizes_mean_onestep, sizes_std_onestep, sizes_mean_twostep,sizes_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Ranges_Sum', rsum_mean_onestep, rsum_std_onestep, rsum_mean_twostep,rsum_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    pattern_row = ['Coverage', coverage_mean_onestep, coverage_std_onestep, coverage_mean_twostep,coverage_std_twostep]
    all_metrics_mean_df.loc[len(all_metrics_mean_df), :] = pattern_row

    metrics_dataframes.append(all_metrics_mean_df)

    display(all_metrics_mean_df)

p = 0.5


,Metric,Onestep_MEAN,Onestep_STD,Twostep_MEAN,Twostep_STD
0,Time,0.985347,0.246207,1.82067,0.636175
1,Size,15.79661,2.880558,15.79661,2.880558
2,Ranges_Sum,12.343276,1.483785,12.56332,1.475911
3,Coverage,1.0,0.0,1.0,0.0


In [11]:
# dfi.export(all_metrics_mean_df, './saved_dataframe_image/'+dataset_name +"_"+ str(p_values[0])+'.jpg')
# all_metrics_mean_df.to_csv('./saved_dataframe_csv/'+dataset_name +"_"+ str(p_values[0])+'.csv',index=False)